# Examen d'un modèle Basenji

In [1]:
import json
import numpy as np

In [12]:
import tensorflow as tf

In [2]:
from basenji import dataset
from basenji import seqnn
from basenji import trainer

In [3]:
params_file = "/home/bureau/projects/def-bureau/basenji/manuscripts/akita/params.json"

In [4]:
data_stats_file = "/home/bureau/projects/def-bureau/bureau/distiller/results_Neu10000/data/1m/statistics.json"

In [14]:
tfr_train_full = "/home/bureau/projects/def-bureau/bureau/distiller/results_Neu10000/data/1m/tfrecords/train-*.tfr"
tfr_eval_full = "/home/bureau/projects/def-bureau/bureau/distiller/results_Neu10000/data/1m/tfrecords/valid-*.tfr"

## Chargement des paramètres du modèle

In [19]:
with open(params_file) as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']
params_model['batch_norm'] = False
params_model['head_hic'][-1]['units'] =1

## Chargement des stats du modèle

In [9]:
with open(data_stats_file) as data_stats_open:
    data_stats = json.load(data_stats_open)
data_stats

{'num_targets': 1,
 'train_seqs': 9211,
 'valid_seqs': 7549,
 'test_seqs': 7564,
 'seq_length': 1048576,
 'pool_width': 2048,
 'crop_bp': 65536,
 'diagonal_offset': 2,
 'target_length': 99681}

## Chargement des données

In [15]:
train_data = dataset.SeqDataset(tfr_train_full,
    params_train['batch_size'],
    data_stats['seq_length'],
    data_stats['target_length'],
    tf.estimator.ModeKeys.TRAIN)
eval_data = dataset.SeqDataset(tfr_eval_full,
    params_train['batch_size'],
    data_stats['seq_length'],
    data_stats['target_length'],
    tf.estimator.ModeKeys.EVAL)


/home/bureau/projects/def-bureau/bureau/distiller/results_Neu10000/data/1m/tfrecords/train-*.tfr has 9211 sequences with 0/1 targets
/home/bureau/projects/def-bureau/bureau/distiller/results_Neu10000/data/1m/tfrecords/valid-*.tfr has 7549 sequences with 0/1 targets


## Initialisation du modèle

In [20]:
seqnn_model = seqnn.SeqNN(params_model)

AttributeError: in user code:

    /lustre03/project/6000443/basenji/basenji/layers.py:496 call  *
        seq_len = inputs.shape[1].value

    AttributeError: 'int' object has no attribute 'value'
